In [1]:
!pip install groq

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!export GROQ_API_KEY=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

In [26]:
import os
import json
import pandas as pd
from groq import Groq
from tqdm.auto import tqdm

client = Groq(api_key='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') 
df = pd.read_csv('misclassified_samples/ms_cg.csv')

sentences = df['sentence'].tolist()
labels = df['label'].tolist()

incorrect_predictions = [1 - int(label) for label in labels]  
def save_to_json(data, file_name='toxic_not_profane_sentences.json'):
    """Save data to a JSON file."""
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

results = []

num_samples = 50  
for idx in tqdm(range(num_samples), desc="Processing Samples"):

    if idx >= len(sentences):
        print(f"Reached the end of the dataset at index {idx}.")
        break

    sentence = sentences[idx]
    incorrect_prediction = incorrect_predictions[idx]

    prompt = f"""
    The task is to analyze the following sentence and its incorrect classification as either normal (0) or toxic (1). Your objective is to identify the specific words in the sentence that may have contributed to this misclassification. The output should be in the following JSON format:

    ```
    {{
        "sentence": "{sentence}",
        "incorrect_prediction": {incorrect_prediction},
        "words": [w1, w2, ...]
    }}
    ```

    - List only the relevant words that influenced the incorrect prediction.
    - Use without any additional text, numbers, explanations, or punctuation.
    """

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama3-8b-8192"  # Ensure this is the correct model name
        )

        response = chat_completion.choices[0].message.content.strip()

        try:
            response_json = json.loads(response)
            words = ','.join(response_json.get("words", []))
        except json.JSONDecodeError:
            words_line = next((line for line in response.split('\n') if line.strip().startswith('"words":')), None)
            if words_line:
                start = words_line.find('[') + 1
                end = words_line.find(']')
                words = words_line[start:end].replace('"', '').replace(' ', '')
            else:
                words = ""

        results.append({
            "sentence": sentence,
            "incorrect_prediction": incorrect_prediction,
            "words": words
        })

    except Exception as e:
        print(f"Error processing sentence {idx}: {e}")
        continue

    if (idx + 1) % 10 == 0:
        save_to_json(results, file_name='misclassification_word_explanations.json')
        print(f"Saved {len(results)} records to 'misclassification_word_explanations.json'")

if len(results) % 10 != 0:
    save_to_json(results, file_name='misclassification_word_explanations.json')
    print(f"Saved final {len(results)} records to 'misclassification_word_explanations.json'")

print("Process complete. Results saved to 'misclassification_word_explanations.json'")


Processing Samples:   0%|          | 0/50 [00:00<?, ?it/s]

Saved 10 records to 'misclassification_word_explanations.json'
Saved 20 records to 'misclassification_word_explanations.json'
Saved 30 records to 'misclassification_word_explanations.json'
Saved 40 records to 'misclassification_word_explanations.json'
Saved 50 records to 'misclassification_word_explanations.json'
Process complete. Results saved to 'misclassification_word_explanations.json'


In [28]:
import json

def extract_all_words(file_path):
    all_words = []

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Iterate through each entry and extract words
    for entry in data:
        words_str = entry.get('words', '')
        # Split the words by comma
        words = words_str.split(',')
        # Strip any leading/trailing whitespace and add to the list
        words = [word.strip() for word in words if word.strip()]
        all_words.extend(words)

    return all_words

# Path to your JSON file
json_file_path = 'misclassification_word_explanations.json'

# Extract all words
all_words = extract_all_words(json_file_path)

# Optionally, to get unique words, uncomment the following line:
# all_words = list(set(all_words))

print("List of All Words:")
print(all_words)

# If you want to save the list to a file, you can do so as follows:
with open('all_words_list.txt', 'w', encoding='utf-8') as f:
    for word in all_words:
        f.write(f"{word}\n")

print(f"\nAll words have been saved to 'all_words_list.txt'")


List of All Words:
['beaner', 'drunk', '😂', '😂', '😂', 'raped', 'retarded', 'pakistani', 'paki', 'divorced', 'married', 'dyke', 'white', 'trash', 'collapsing', 'faggot', 'heeb', 'muramasa', 'jihadi', 'nigger', 'jewish', 'holocaust', 'negation', 'dykes', 'autistic', 'retarded', 'negro', 'beat', 'james', 'pimped', 'black', 'gook', 'fuck', 'fuck', 'paki', 'raping', 'degeneracy', 'ching', 'chong', 'moslem', 'slave', 'niggers', 'muzzies', 'spics', 'fuck', 'chinks', 'niggers', 'redskins', 'racist', 'ugly', 'dykes', 'moslems', 'dhimmi', 'un', 'moslem', 'niggerz', 'anti', 'gay', 'bigot', 'faggots', 'goy', 'dindu', 'kill', 'white', 'king', 'kike', 'genetic', 'jew', 'white', 'nations', 'dyke', 'dyke', 'big', 'mohajirs', 'slums', 'karachi', 'goons', 'kill', 'mulla', 'kill', 'raped', 'placating', 'shouters', 'anti', 'sheboon', 'ebonics', 'nigga', 'shoot', 'kill', 'lawless', 'shithole', 'peasant', 'coming', 'for', 'your', 'dollars', 'nuclear', 'holocaust', 'nazi', 'puke', 'sacks', 'white suppressiom

In [33]:
import os
import json
from groq import Groq

client = Groq(api_key="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")

def read_words_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        words = file.read().splitlines()
    return words

def get_completion(prompt, model="llama3-8b-8192", temperature=0.0, max_tokens=2000):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model=model,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response

file_path = 'all_words_list.txt'
words = read_words_from_file(file_path)
prompt = f"""Group the following words based on similarity. For each group, provide a descriptive label that accurately captures the common theme among the words. The groups should be as distinct as possible. After grouping, provide both the group label and the list of words for each group. Place all the words of dictionary in at least 1 group and if they don't fall in any group put them in others.

Words:
{', '.join(words)}

Your output should be in this format:

1. **[Race]**: word1, word2, word3, ...
2. **[Religion]**: word1, word2, word3, ...
3. **[Gender]**: word1, word2, word3, ...
"""
response = get_completion(prompt)
# choices[0].message.content.strip()
content = response.choices[0].message.content.strip()
with open('grouped_words_output.txt', 'w', encoding='utf-8') as f:
    f.write(content)


In [ ]:
import shutil
import os

def zip_final_output(output_dir='final_output', output_filename='final_output.zip'):
    """
    Compresses the specified output directory into a ZIP archive.

    Parameters:
    - output_dir (str): Path to the directory to compress.
    - output_filename (str): Desired name for the ZIP file.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        print(f"Error: The directory '{output_dir}' does not exist.")
        return

    # Create a ZIP archive
    shutil.make_archive(base_name=output_filename.replace('.zip', ''), format='zip', root_dir=output_dir)
    print(f"Successfully created '{output_filename}' containing the contents of '{output_dir}'.")

if __name__ == "__main__":
    zip_final_output()
